# Merging Binary Black Holes from Wolf-Rayet-Black Hole progenitors

*Puggioni Dario, Salvador Alberto, Saran Gattorno Giancarlo, Volpi Gaia*

## Pre-processing of the dataset

In the initial part of this project, our aim is to clean the datasets by removing all the informations that will be unnecessary for further analysis. Our final goal is to determine the conditions causing a black hole - Wolf-Rayet binary system to evolve into binary black holes (BBH) that merge via emission of graviational waves. To achieve this, the main steps we will undertake are:


- Load the dataset containing the evolutionary and initial conditions of the systems.
- Remove all the labels that are irrelevant for our analysis.
- Select the systems that, at a certian point in their evolution, become Wolf-Rayet - black hole binaries.
- Identify which of these systems undergo mass transfer proccesses.
- Determine which of these systems evolve into BBHs.
- Assess which of these BBHs systems end up merging via gravitational waves emission.

We choose to employ the Python open-source library Dask for this task. Given that the dataset we are working with is too large to fit in memory, Dask is an optimal solution as it is designed for parallel computing, enabling Python code to scale across multiple multi-core machines.\
The parallelization of the computing process is performed on a cluster of virtual machines provided by the cloud service CloudVeneto.

--------

### 1. Setting up the cluster


We set up a cluster on CloudVeneto composed of four VMs: `binary01` at IP `10.67.22.174`, `binary02` at IP `10.67.22.251`, `binary03` at IP `10.67.22.93` and `binaryNFServer` at IP `10.67.22.36`. The dataset is stored in a CloudVeneto volume mounted on `binaryNFServer` and shared between the VMs via NFS. The three VMs that are NFS clients are used for building the dask cluster in charge of the processing task.
`binary01` is the scheduler. `binary01`, `binary02`, `binary03` all act as workers.


We primarily work with Dask DataFrame, a high-level collection that enables parallelization of DataFrame-based workloads. A Dask DataFrame consists of many smaller Pandas DataFrames, partitioned along the index.

In [ ]:
#importing all the necessary libraries
import pandas as pd 
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client, SSHCluster
from dask import delayed, compute
import time
import os
import glob
import gc 

In [ ]:
def init_client(nthreads=4, nworkers=1):
    """Initialize the dask client.

    Args:
        nthreads (int): Number of utilized threads per host, maximum and default 4
        nworkers (int): Number of dask workers per host
    """ 
    #shutdown all existing Client instances
    for obj in gc.get_objects():
        if isinstance(obj, Client):
            obj.shutdown()
        
    #Instantiate cluster with specified configs
    cluster = SSHCluster(
        ["binary01", "binary01", "binary02", "binary03"],
        connect_options={"known_hosts":None, 'password': 'password'}, #insert the user's password
        worker_options={"nthreads": nthreads, "n_workers": nworkers},
        scheduler_options={"port": 9876, "dashboard_address": ":7977"}
    )
    
    return Client(cluster)

### 2. The dataset

#### 2.1 SEVN

The dataset consists of two outputs computed at different metallicities (Z=0.0014 and Z=0.02) by the population synthesis code SEVN (Stellar EVolution for N-body). We work with binary systems whose evolution is described by SEVN including the following processes: wind mass transfer, Roche-lobe overflow (RLO), common envelope (CE), stellar tides, circularization at the RLO onset, collision at periastron, orbit decay by GW emission, and stellar mergers.

SEVN uses a prediction-correction method to adapt the time-step accounting for the large physical range of timescales typical of stellar and binary evolution. To decide the time-step, it looks at a sub-set of stellar and binary properties: if any of them changes too much during a time-step, it reduces the time-step and repeats the calculation. A special treatment is used when a star approaches a change of phase to guarantee that the stellar properties are evaluated just after and before the change of phase. 

#### 2.2 Structure of the dataset
The datasets are contained in one folder each, named after their metallicity. Inside each folder we find: 
- For Z=0.02: a single file named `output_0.csv`
- For Z=0.00014: 40 files named `output_*.csv` where * is a placeholder for numbers between 0 and 39

In addition, we have information about the initial conditions of each system considered. This information is contained in the folder `initial`, which includes files of the type `evolved_*.dat`. Output files and evolved files have a one-to-one correspondence. For example, the initial conditions of the systems in `output_0.csv` are present in `evolved_0.dat`. The link between the two is through the 'ID' label, which uniquely identifies a system.

We built a function able to retrieve the path of the folders related to a specific metallicity, the one containing the `output_*.csv` files and the one containing the corresponing `evolved_*.dat` files. This ensures our code is flexible and capable to adapt to potential new data.

In [ ]:
def grab_paths(main_folder, subfolder_index):
    """Grab the directories of ouputs and initial configurations at a given index in the list of available metallicities.
    
    Args:
        main_folder (str): Path to the dataset
        subfolder_index (int): Index in the list of available metallicities
    """
    subfolders = glob.glob(main_folder) #Folders named with numbers, so FS always orders them before initial
    out_folder = subfolders[subfolder_index]
    init_folder = glob.glob(subfolders[-1] + '/*')[subfolder_index]
    
    return out_folder, init_folder

In the following, we report an example of the structure of the informations contained in an 'output' type file:

In [ ]:
path = './demo_Z0.00014/output_0.csv' # local path of the dataset at Z = 0.02
df = dd.read_csv(path, dtype={'BEvent': float})
df.head(5)

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ID</th>
      <th>name</th>
      <th>Mass_0</th>
      <th>Radius_0</th>
      <th>Phase_0</th>
      <th>PhaseBSE_0</th>
      <th>RemnantType_0</th>
      <th>Hsup_0</th>
      <th>Mass_1</th>
      <th>Radius_1</th>
      <th>Phase_1</th>
      <th>PhaseBSE_1</th>
      <th>RemnantType_1</th>
      <th>Hsup_1</th>
      <th>Semimajor</th>
      <th>Eccentricity</th>
      <th>BWorldtime</th>
      <th>Period</th>
      <th>GWtime</th>
      <th>RL0</th>
      <th>RL1</th>
      <th>BEvent</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0</td>
      <td>144668535680303</td>
      <td>1.208462</td>
      <td>0.000016</td>
      <td>7</td>
      <td>13.0</td>
      <td>5</td>
      <td>0.0</td>
      <td>11.84389</td>
      <td>3.936276</td>
      <td>1</td>
      <td>1.0</td>
      <td>0</td>
      <td>0.0</td>
      <td>139.0903</td>
      <td>0.437699</td>
      <td>19.30851</td>
      <td>0.143997</td>
      <td>140246700.0</td>
      <td>28.92884</td>
      <td>80.19596</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0</td>
      <td>144668535680303</td>
      <td>1.208462</td>
      <td>0.000016</td>
      <td>7</td>
      <td>13.0</td>
      <td>5</td>
      <td>0.0</td>
      <td>11.84389</td>
      <td>3.936276</td>
      <td>1</td>
      <td>1.0</td>
      <td>0</td>
      <td>0.0</td>
      <td>139.0903</td>
      <td>0.437699</td>
      <td>19.30851</td>
      <td>0.143997</td>
      <td>140246700.0</td>
      <td>28.92884</td>
      <td>80.19596</td>
      <td>-1.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0</td>
      <td>144668535680303</td>
      <td>1.208462</td>
      <td>0.000016</td>
      <td>7</td>
      <td>13.0</td>
      <td>5</td>
      <td>0.0</td>
      <td>11.84374</td>
      <td>4.250581</td>
      <td>1</td>
      <td>1.0</td>
      <td>0</td>
      <td>0.0</td>
      <td>139.0920</td>
      <td>0.437699</td>
      <td>20.46043</td>
      <td>0.144001</td>
      <td>140256600.0</td>
      <td>28.92928</td>
      <td>80.19674</td>
      <td>-1.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0</td>
      <td>144668535680303</td>
      <td>1.208462</td>
      <td>0.000016</td>
      <td>7</td>
      <td>13.0</td>
      <td>5</td>
      <td>0.0</td>
      <td>11.84363</td>
      <td>4.513947</td>
      <td>1</td>
      <td>1.0</td>
      <td>0</td>
      <td>0.0</td>
      <td>139.0932</td>
      <td>0.437699</td>
      <td>21.23934</td>
      <td>0.144003</td>
      <td>140264000.0</td>
      <td>28.92962</td>
      <td>80.19733</td>
      <td>-1.0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>0</td>
      <td>144668535680303</td>
      <td>1.208462</td>
      <td>0.000016</td>
      <td>7</td>
      <td>13.0</td>
      <td>5</td>
      <td>0.0</td>
      <td>11.84353</td>
      <td>4.752590</td>
      <td>1</td>
      <td>1.0</td>
      <td>0</td>
      <td>0.0</td>
      <td>139.0942</td>
      <td>0.437699</td>
      <td>21.90684</td>
      <td>0.144005</td>
      <td>140270400.0</td>
      <td>28.92990</td>
      <td>80.19784</td>
      <td>-1.0</td>
    </tr>
  </tbody>
</table>
</div>

### 3. Selecting systems

The goal of this section is to identify systems that transition through a phase involving a black hole and a Wolf-Rayet star, form a binary black hole (BBH), and end up merging via emission of gravitational waves within a Hubble time. 

X-ray binaries containing a black hole and a massive star are among the most promising observational candidates for BBH progenitors. Mass transfer events play a crucial role in shrinking the orbit of these systems, facilitating their eventual merger. In particular, we focus on Wolf-Rayet stars as they form as a result of significant mass transfer and strong stellar winds that strip away the hydrogen envelope of a massive star. Under the right conditions, a binary system can evolve into a black hole – Wolf-Rayet star system. Such systems may become visible as X-ray binaries because the intense stellar winds can lead to the formation of an accretion disk around the black hole, producing powerful X-ray emissions.

<span style="color:blue">Io da qui in poi presenterei come soggetto principale la funzione output analysis, spiegando le varie operazioni che fa (cioè quello che c'è scritto nei sottoparagrafi qui sotto)</span>

#### 3.1 Wolf-Rayet - black hole progenitors 

First, we select the IDs of the systems that, at a certain time step during their evolution, consist of a Wolf-Rayet star (PhaseBSE={7, 8}) and a black hole (PhaseBSE=14). Additionally, we select and save in `df_WRBH` the features we'll use in our analysis: _ID_, _Mass_0_, _Mass_1_, _Semimajor_, and _Eccentricity_. These features correspond to the parameters at the formation of the WR-BH binary.

#### 3.2 Mass transfer events

Population-synthesis studies suggest that at least one mass transfer episode is necessary to shrink the orbit of the progenitor binary star and allow the coalescence of the two final black holes within a Hubble time. **Keeping this in mind, we decide to consider only systems that undergo mass transfer events.** We count these events for each of the previously selected systems, categorizing them into stable and unstable (_MTEvents_stable_ and _MTEvents_unstable_).

##### 3.2.1 Roche Lobe overflow

The effective potential in a binary system, which includes the gravitational potential of both stars and the centrifugal force acting on a mass-less test particle, is known as the Roche-Lobe potential. This potential has five Lagrangian points where its gradient is zero. The innermost of these points is called L1, and the equipotential surface passing through L1 connects the gravitational spheres of influence of the two stars. When one star fills its Roche lobe, matter can flow through the L1 point into the Roche lobe of the other star. This process, known as Roche-lobe overflow (RLO), is the primary mechanism for mass transfer between stars in a binary system.

In the case of stable, (quasi-)conservative mass transfer, most, but not necessarily all, of the transferred mass is accreted by the companion star, generally leading to a widening of the binary. This process continues until most of the hydrogen-rich envelope of the donor star has either been transferred to the companion or lost from the system. To count the number of stable mass transfer events, we consider the IDs corresponding to the end of an RLO event (BEvent=5).

##### 3.2.2 Common Envelope

Mass transfer becomes unstable when the accreting star cannot absorb all the material transferred from the donor star. In this scenario, the excess material accumulates on the accretor, causing it to expand and eventually fill and overfill its Roche lobe. This leads to the formation of a common-envelope (CE) system, where the core of the donor star and the companion star are immersed in the donor's envelope. Once a CE system forms, friction between the binary components and the surrounding envelope causes the stars to spiral closer together. This process continues until enough orbital energy has been released to eject the envelope. This mechanism is believed to be the primary way an initially wide binary can evolve into a very close binary. To count the number of unstable mass transfer events, we consider the IDs corresponding to the start of a CE (BEvent=7) or the start of an unstable RLO ending in a CE (BEvent=11). 

It is crucial to say that we exclude any mass transfer event that directly results in the merging of the two stars, as these would not become gravitational wave merging BBHs. 

#### 3.3 Binary black holes

Next, we select the IDs of BBHs (PhaseBSE=14) that merge via gravitational waves emission. To do so we reqiure that the sum of _GWtime_ (GW orbital decay time in Myr) and _BWorldtime_ (time elapsed in the simulations) is smaller than 14 billions years, the Hubble time.

In SEVN _GWtime_ is defined as:
\begin{equation}
\frac{t_{GW}}{1+f_{corr}(e)}
\end{equation}

where $t_{GW}=\frac{5}{26} \frac{c^5}{G^3} \frac{a^4}{M_1 M_2 (M_1+M_2)} (1-e^2)^{\frac{7}{5}}$ accounts for obrital decay and circularization by GWs, and $f_{corr}(e)$ is a correction term at large eccentricities.

#### 3.4 Initial conditions

Finally, we add to the filtered systems some of their initial conditions: the masses of the two stars, eccentricity, and semi-major axis.

In [ ]:
def output_analysis(subfolders, record_index, n_partitions=None):
    """Process a single couple of output_*.csv and evolved_*.csv files at a given metallicity.
    
    Args:
        n_partitions (int): Number of partitions per output file
        subfolders (list):  Paths to the respective folders containing the file couple
        record_index (int): Index of the file couple
    """

    # we first remove some columns 
    col_to_remove = ['name', 'Radius_0', 'Phase_0', 'RemnantType_0', 'Hsup_0', 'Radius_1', 'Phase_1',
                 'RemnantType_1', 'Hsup_1', 'Period', 'RL0', 'RL1']
    
    out_folder, init_folder = subfolders
    out_path = f'{out_folder}/output_{record_index}.csv'
    init_path = f'{init_folder}/evolved_{record_index}.dat'
    if n_partitions==None:
        df = dd.read_csv(out_path, dtype={'BEvent': float}).\
            drop(columns=col_to_remove)
    else: 
        df = dd.read_csv(out_path, dtype={'BEvent': float}).\
            drop(columns=col_to_remove).repartition(npartitions=n_partitions) 
    

    ####### WR-BH SELECTION
    cond_WRBH = (((df['PhaseBSE_0'] == 8) | (df['PhaseBSE_0'] == 7)) & (df['PhaseBSE_1'] == 14.0)) | \
                (((df['PhaseBSE_1'] == 8) | (df['PhaseBSE_1'] == 7)) & (df['PhaseBSE_0'] == 14.0))

    df_WRBH = df[cond_WRBH][['ID', 'Mass_0', 'Mass_1', 'Semimajor', 'Eccentricity', 'BEvent']]
    
    
    ####### MASS TRANSFER EVENTS
    n_stable = df_WRBH[df_WRBH['BEvent'] == 5][['ID', 'BEvent']].groupby('ID').size().reset_index().rename(columns={0: 'MTEvents_stable'})
    n_unstable = df_WRBH[df_WRBH['BEvent'].isin([7, 11])][['ID', 'BEvent']].groupby('ID').\
                    size().reset_index().rename(columns={0: 'MTEvents_unstable'})
    mass_transfer = n_stable.merge(n_unstable, how='outer', on='ID')
    mass_transfer['MTEvents_unstable'] = mass_transfer['MTEvents_unstable'].fillna(0).astype(int) 
    mass_transfer['MTEvents_stable'] = mass_transfer['MTEvents_stable'].fillna(0).astype(int)


    df_WRBH = df_WRBH.drop_duplicates(subset='ID') # keeping only the first row for each ID
    data = mass_transfer.merge(df_WRBH, how='inner', on='ID')

    '''
    se volessimo aggiungere tutti i sistemi WR-BH
    
    data = mass_transfer.merge(df_WRBH, how='outer', on='ID')
    data['MTEvents_unstable'] = mass_transfer['MTEvents_unstable'].fillna(0).astype(int) 
    data['MTEvents_stable'] = mass_transfer['MTEvents_stable'].fillna(0).astype(int)
    '''

    ####### MERGING BBHs SELECTION
    cond_GW = (df['PhaseBSE_0'] == 14.0) & (df['PhaseBSE_1'] == 14.0) & \
              ((df['GWtime'] + df['BWorldtime']) < int(14e+03))
    id_GW = df[cond_GW]['ID'].drop_duplicates() 
    
    ###### WR-BH BECOMING MERGINING BH-BH SECTION 
    id_GW = id_GW.to_frame(name='ID').assign(Merge=1)
    data = data.merge(id_GW, how='left', on='ID').fillna({'Merge': 0})

    ###### INITIAL CONDITIONS
    df_initial = dd.read_csv(init_path, sep='\s+')
    df_initial = df_initial[["#ID", "Mass_0", "Mass_1", "a", "e"]]
    # renaming
    df_initial = df_initial.rename(columns={"#ID": "ID"}) 
    df_initial = df_initial.rename(columns={"Mass_0": "Mass_0_initial"})   
    df_initial = df_initial.rename(columns={"Mass_1": "Mass_1_initial"})
    data = data.merge(df_initial, how='inner', on='ID') 
    
    return data

In [ ]:
def folder_analysis(folder_idx, n_partitions=None, main_path='/mnt/data/*'):
    """Process all the files in a folder of a given metallicity.

    Args:
        main_path (str): Path to the dataset
        n_partitions (int): Number of partitions per dask dataframe loading a single output file
        folder_idx (int): Index in the list of available metallicities
    
    """
    
    # Process each file using the provided analysis function
    data_list = []
    subfolders = grab_paths(main_path, folder_idx)
    n_files = len(os.listdir(subfolders[1])) #number of files in folder

    
    #Iterate all files in the folder
    for index_file in range(n_files):
        data = output_analysis(subfolders=subfolders, record_index=index_file, n_partitions=n_partitions)
        
        data_list.append(data)
        
    dd_data = dd.concat(data_list)

    return dd_data

### 4. The Output

The final output consists of two dataframes, `out0` and `out1`, corresponding to metallicities Z=0.02 and Z=0.00014, respectively. These dataframes contain information on Wolf-Rayet - black hole binaries that experience at least one event of mass transfer event that does not direcltly lead to the merging of the two. The column _Merge_ indicates whether these systems eventually evolve into BBHs that merge through the emission of gravitational waves.

In [ ]:
client = init_client()

# Z = 0.02
data = folder_analysis(folder_idx=1).compute()
data.to_csv('./out/out0.csv', index=False)

# Z = 0.00014
data = folder_analysis(folder_idx=0).compute()
data.to_csv('./out/out1.csv', index=False) 